In [2]:
import pandas as pd
import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import re # for regular expressions
import pandas as pd 
pd.set_option("display.max_colwidth", 200) 
import string
import nltk # for text manipulation
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
#from gensim.models.deprecated.doc2vec import LabeledSentence
import gensim
from sklearn.linear_model import LogisticRegression
from scipy import stats 
from sklearn import metrics 
from sklearn.metrics import mean_squared_error,mean_absolute_error, make_scorer,classification_report,confusion_matrix,accuracy_score,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.naive_bayes import BernoulliNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import warnings 
warnings.filterwarnings("ignore")

%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import common
from common import *

In [3]:
DocumentList = pd.read_csv("Copy of Coronavirus Tweets - Copy.csv", encoding='latin', usecols=['ID', 'Description',
                                                                                                   'Sentiment',
                                                                                                   'IsRelevant',
                                                                                                   'tfVector'])


In [4]:
def cleantweets(text):
    text=str(text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r'[^a-zA-Z#]+', ' ',text)
    text = text.lower()
    return text

In [5]:
DocumentList['Description']=DocumentList['Description'].apply(lambda x:cleantweets(x))
#remove short words
DocumentList['Description']=DocumentList['Description'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2]))

In [6]:
#nltk pakage
tokenizer = TweetTokenizer(preserve_case=False, 
                           strip_handles=True,
                           reduce_len=True)

DocumentList['Des_token']=DocumentList['Description'].apply(lambda x:tokenizer.tokenize(x))

In [7]:
def tokenize(sentence):
    token = []
    for word in CountVectorizer().build_tokenizer()(sentence):
        token.append(word)
    return token

In [8]:
DocumentList['Des_token']=DocumentList['Description'].apply(lambda x:tokenize(x))

In [9]:
stopwords_english = stopwords.words('english')
def cleanstopword(tweet_tokens):
  tweets_clean = []
  for word in tweet_tokens:
    if (word not in stopwords_english and word not in string.punctuation):
      tweets_clean.append(''.join(word))
  return tweets_clean
##########################################3  

In [10]:
DocumentList['Des_token']=DocumentList['Des_token'].apply(lambda x:cleanstopword(x))

In [11]:
def getAllTerms(textList):
    terms = []
    for i in range(len(textList)):
        for j in range(len(textList[i])):
            terms.append(textList[i][j])
    return sorted(set(terms))

In [12]:
terms = getAllTerms(DocumentList['Des_token'])

In [17]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
  
warnings.filterwarnings(action = 'ignore')
  
import gensim
from gensim.models import Word2Vec
  

  

# Create CBOW model
model1 = gensim.models.Word2Vec(terms, vector_size=300, min_count=2,window=5, sg=1,workers=4)
  


In [18]:
model1

In [26]:
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens)<1:
        return np.zeros(300)
    else:
        for tok in doc_tokens:
            if tok in model1.wv.index_to_key:
                embeddings.append(model1.wv.word_vec(tok))
            else:
                embeddings.append(np.random.rand(300))
        # mean the vectors of individual words to get the vector of the document
        return np.mean(embeddings, axis=0)

In [29]:
queries=['covid','vaccine']


In [30]:
vector=get_embedding_w2v(queries)

In [31]:
vector

array([0.53551828, 0.41854745, 0.81609005, 0.57266972, 0.50472997,
       0.41348212, 0.53923726, 0.80465376, 0.40086566, 0.07424934,
       0.56985353, 0.24252448, 0.29681483, 0.35238451, 0.61461851,
       0.8376321 , 0.36393108, 0.35544147, 0.54156872, 0.66569567,
       0.58098772, 0.23273798, 0.13667955, 0.38882995, 0.41564568,
       0.42368113, 0.71667582, 0.47430166, 0.58305173, 0.67677658,
       0.64775851, 0.78045972, 0.60619613, 0.64312217, 0.8680184 ,
       0.53780659, 0.40217122, 0.5234382 , 0.39456944, 0.33067418,
       0.29526498, 0.21867255, 0.5303914 , 0.34551042, 0.17257685,
       0.44004578, 0.5729511 , 0.49387241, 0.3585988 , 0.07662811,
       0.37232853, 0.26929815, 0.38815603, 0.29538079, 0.29766316,
       0.24519978, 0.69674651, 0.40886687, 0.36303954, 0.06569451,
       0.49796969, 0.22557687, 0.63701466, 0.57208601, 0.68632729,
       0.19848978, 0.37457135, 0.50427379, 0.67631295, 0.51261446,
       0.9890376 , 0.6837944 , 0.36745047, 0.3048796 , 0.37300

In [34]:
DocumentList['vector']=DocumentList['Des_token'].apply(lambda x:get_embedding_w2v(x))

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
DocumentList['similarity']=DocumentList['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
DocumentList.sort_values(by='similarity',ascending=False,inplace=True)
DocumentList.head(10).reset_index(drop=True)

,Description,Sentiment,IsRelevant,ID,tfVector,Des_token,vector,similarity
0,too work retail the store where would get orders from the drivers know have told them work store near there store have been told day day basis roughly item,Neutral,0,27369,NaN,"[work, retail, store, would, get, orders, drivers, know, told, work, store, near, store, told, day, day, basis, roughly, item]","[0.5124790327156227, 0.4361002119303823, 0.598646048878666, 0.5733125012590152, 0.6037498071939685, 0.5274970532365403, 0.4721870527564337, 0.5475916829654918, 0.5308112906366967, 0.48899337409897...",0.918534
1,day covid diary all schools are shut friday meanwhile braved the supermarket armed with carrier bags and hand sanitizer the shelves where pretty empty also saw elderly gent wearing welding mask wh...,Positive,1,5035,NaN,"[day, covid, diary, schools, shut, friday, meanwhile, braved, supermarket, armed, carrier, bags, hand, sanitizer, shelves, pretty, empty, also, saw, elderly, gent, wearing, welding, mask, bizarre]","[0.5704910972802842, 0.42575185715042246, 0.4029838927139648, 0.4468534234028975, 0.5703589785802079, 0.4997081231912942, 0.5690486378334831, 0.5628777879607043, 0.5074086862907985, 0.479480671662...",0.918260
2,panic buying reaction coronavirus has meant drop donations for some foodbanks while some are also struggling buy supplies from grocery stores feeding america establishes covid response fund help f...,Negative,0,598,NaN,"[panic, buying, reaction, coronavirus, meant, drop, donations, foodbanks, also, struggling, buy, supplies, grocery, stores, feeding, america, establishes, covid, response, fund, help, food, banks,...","[0.40902034422222444, 0.48297367409893754, 0.5720102381228351, 0.5024366942754132, 0.5595574700839206, 0.48308097911771897, 0.5265511825441006, 0.5386432218716487, 0.5678654419920186, 0.4369796631...",0.918055
3,can help but wonder the one third people saw the grocery store wearing masks are being forced use them after covid patients cough their face too,Negative,0,34612,NaN,"[help, wonder, one, third, people, saw, grocery, store, wearing, masks, forced, use, covid, patients, cough, face]","[0.5469317265237348, 0.43453320213120455, 0.5551190337066506, 0.5392189638575862, 0.4937792399801246, 0.47348353336298626, 0.6429286203435995, 0.47986935553837523, 0.48194369694910566, 0.479314593...",0.917843
4,these unprecedented times calls unprecedented measures caf india appeals you support reach out the vulnerable people with urgent food supplies hygiene kits and accurate information about preventio...,Negative,0,15272,NaN,"[unprecedented, times, calls, unprecedented, measures, caf, india, appeals, support, reach, vulnerable, people, urgent, food, supplies, hygiene, kits, accurate, information, prevention, covid, ave...","[0.5291587559496079, 0.5223125262482532, 0.5753982354647033, 0.5025361869393413, 0.560602881778188, 0.5467917188106997, 0.5087006598894713, 0.45608143839640897, 0.43031883293045964, 0.445982860158...",0.917647
5,come this the white house now advising everyone not head the grocery store pharmacy the coming two weeks the next two weeks are extraordinarily important white house response coordinator deborah b...,Positive,1,29317,NaN,"[come, white, house, advising, everyone, head, grocery, store, pharmacy, coming, two, weeks, next, two, weeks, extraordinarily, important, white, house, response, coordinator, deborah, birx, said]","[0.47007189003721184, 0.4204719778934094, 0.5015609020480467, 0.4885943592785753, 0.44131183004226, 0.5371532611159601, 0.49100569516295794, 0.5622903665518074, 0.4844938677326825, 0.5696832431299...",0.917431
6,japan got great food mascots amp check out this one spotted month back haneda amp perhaps worth buying you worried about hygiene amp shortages the age,Positive,1,16968,NaN,"[japan, got, great, food, mascots, amp, check, one, spotted, month, back, haneda, amp, perhaps, worth, buying, worried, hygiene, amp, shortages, age]","[0.545791345311021, 0.5603386347505851, 0.45267016